In [1]:
pip install fedml-azure --force-reinstall

  Using cached fedml_azure-2.0.1-py3-none-any.whl (30 kB)
  Using cached hdbcli-2.19.20-cp34-abi3-manylinux1_x86_64.whl (10.8 MB)
  Using cached ruamel.yaml-0.18.5-py3-none-any.whl (116 kB)
  Using cached ruamel.yaml.clib-0.2.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (526 kB)
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.19.20
    Uninstalling hdbcli-2.19.20:
      Successfully uninstalled hdbcli-2.19.20
  Attempting uninstall: ruamel.yaml.clib
    Found existing installation: ruamel.yaml.clib 0.2.8
    Uninstalling ruamel.yaml.clib-0.2.8:
      Successfully uninstalled ruamel.yaml.clib-0.2.8
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.18.5
    Uninstalling ruamel.yaml-0.18.5:
      Successfully uninstalled ruamel.yaml-0.18.5
  Attempting uninstall: fedml-azure
    Found existing installation: fedml-azure 2.0.1
    Uninstalling fedml-azure-2.0.1:
      Successfully uninstalled fed

In [2]:
from fedml_azure import create_workspace
from fedml_azure import create_compute
from fedml_azure import create_environment
from fedml_azure import DwcAzureTrain
from fedml_azure import DbConnection
from fedml_azure import register_model,deploy,predict

In [3]:
#creation of workspace
workspace=create_workspace(
    workspace_args={
    "subscription_id": "2759153f-728a-4202-9d26-dfbbf4762c83",
    "resource_group": "fr_retail_sourcegp",
    "workspace_name": "fr_ml_workspace"
    }
)

2023-12-15 11:22:01,594: fedml_azure.logger INFO: Getting existing Workspace


In [4]:
#creation of compute target
compute=create_compute(
    workspace=workspace,
    compute_type='AmlComputeInstance',
    compute_args={
        'vm_size':"Standard_D3_v2",
        'compute_name':'def-compute'
    }
)

2023-12-15 11:22:07,257: fedml_azure.logger INFO: Creating Compute_target.
2023-12-15 11:22:10,690: fedml_azure.logger INFO: Found existing instance, using it.


In [5]:
environment=create_environment(
    workspace=workspace,
    environment_type='CondaSpecificationEnvironment',
    environment_args={
        'name':'conda-env-sp',
        'file_path':'src/conda_dependency.yml'
        }
    )

2023-12-15 11:22:13,054: fedml_azure.logger INFO: Creating Environment.


### Training -

In [6]:
train=DwcAzureTrain(
    workspace=workspace,
    environment=environment,
    experiment_args={'name':'fr-exp'},
    compute=compute
)

2023-12-15 11:22:20,998: fedml_azure.logger INFO: Assigning Workspace.
2023-12-15 11:22:21,000: fedml_azure.logger INFO: Creating Experiment
2023-12-15 11:22:22,276: fedml_azure.logger INFO: Assigning compute.
2023-12-15 11:22:22,277: fedml_azure.logger INFO: Assigning Environment.


In [7]:
#generating the run config
src=train.generate_run_config(
    config_file_path='src/dspconfig.json',
    config_args={
        'source_directory':'Retail-Project',
        'script':'training_script.py',
        'arguments':[
            '--model_file_name','retailmodel.pkl',
            '--dist_table', 'FR_HL_DISTRIBUTOR_V',
            '--dist_size', '1',
            '--product_table', 'FR_HL_PRODUCT_V',
            '--product_size', '1',
            '--retailer_table', 'FR_HL_RETAIL_V',
            '--retailer_size', '1',
            '--retailer1_table', 'FR_HL_RETAILER1_V',
            '--retailer1_size', '1',
            '--retailer2_table', 'FR_HL_RETAILER2_V',
            '--retailer2_size', '1',
            '--retailer3_table', 'FR_HL_RETAILER3_V',
            '--retailer3_size', '1',
            ]
    }
)

2023-12-15 11:22:24,622: fedml_azure.logger INFO: Generating script run config.
2023-12-15 11:22:24,624: fedml_azure.logger INFO: Copying config file for db connection to script_directory Retail-Project


In [8]:
#submitting the training run
run=train.submit_run(src)

2023-12-15 11:22:29,815: fedml_azure.logger INFO: Submitting training run.
RunId: fr-exp_1702639351_c351a9d9
Web View: https://ml.azure.com/runs/fr-exp_1702639351_c351a9d9?wsid=/subscriptions/2759153f-728a-4202-9d26-dfbbf4762c83/resourcegroups/fr_retail_sourcegp/workspaces/fr_ml_workspace&tid=c077bf5f-5cac-4d48-a533-ef49a3a46d93

Streaming user_logs/std_log.txt



********* Handling Data - Splitting into Train and Test *********n

FR_HL_DISTRIBUTOR_V
--- 0.2961282730102539 seconds ---
got dist_data
--- 0.09261536598205566 seconds ---
got product_data
--- 0.024788379669189453 seconds ---
got retailer_data
--- 3.7936267852783203 seconds ---
got retailer3_data
--- 3.8106279373168945 seconds ---
got retailer2_data
--- 3.7077178955078125 seconds ---
got retailer1_data
        productsku  retailer  ...   Material mtd_consumption_lag_1
1022          1061     10001  ...      Denim              0.000000
2860          1061     10001  ...      Denim             10.820318
4956          1061     10

In [9]:
model=train.register_model(
    run=run,
    model_args={
        'model_name':'retail_model',
        'model_path':'outputs/retailmodel.pkl'
        },
        resource_config_args={
            'cpu':1, 'memory_in_gb':0.5
        },
        is_sklearn_model=False
)
print('Name:', model.name)
print('Version:', model.version)

2023-12-15 11:24:01,329: fedml_azure.logger INFO: Registering the model.
Name: retail_model
Version: 3


In [15]:
inference_config_args = {
    'entry_script':'Retail-Project/predict.py',
    'environment': environment
}
deploy_args = {
    'workspace': workspace,
    'name': 'retail-usecase2',
    'models': [model]
}

In [16]:
from fedml_azure import deploy

In [18]:
endpoint_url = deploy(
    compute_type='Local',
    deploy_args=deploy_args,
    inference_config_args=inference_config_args
)

2023-12-15 11:26:29,040: fedml_azure.logger INFO: The parameter 'deploy_config_args' is not specified. An empty configuration object will be used based on the desired target.
Generating Docker build context.
Package creation Succeeded
Logging into Docker registry cd89c64731ef49f189499606c5257fe4.azurecr.io
Logging into Docker registry cd89c64731ef49f189499606c5257fe4.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM cd89c64731ef49f189499606c5257fe4.azurecr.io/azureml/azureml_ba57d16069b1144b9fc17220c37e7d37
 ---> 1a96874603ec
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> c0ec9fde9b11
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjI3NTkxNTNmLTcyOGEtNDIwMi05ZDI2LWRmYmJmNDc2MmM4MyIsInJlc291cmNlR3JvdXBOYW1lIjoiZnJfcmV0YWlsX3NvdXJjZWdwIiwiYWNjb3VudE5hbWUiOiJmcl9tbF93b3Jrc3BhY2UiLCJ3b3Jrc3BhY2VJZCI6ImNkODljNjQ3LTMxZWYtNDlmMS04OTQ5LTk2MDZjNTI1N2ZlNCJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/a

In [19]:
endpoint_url

('http://localhost:32773/score',
 None,
 LocalWebservice(workspace=Workspace.create(name='fr_ml_workspace', subscription_id='2759153f-728a-4202-9d26-dfbbf4762c83', resource_group='fr_retail_sourcegp'), name=retail-usecase2, image_id=None, image_digest=None, compute_type=Local, state=running, scoring_uri=http://localhost:32773/score, tags=None, properties=None, created_by=None))

In [20]:
import json
import pandas as pd
from fedml_azure import DbConnection
from fedml_azure import predict

In [21]:
db = DbConnection(url='src/dspconfig.json')
dist_data = db.get_data_with_headers(table_name='FR_HL_DISTRIBUTOR_V', size=1)
dist_data = pd.DataFrame(dist_data[0], columns=dist_data[1])

product_data = db.get_data_with_headers(table_name='FR_HL_PRODUCT_V', size=1)
product_data = pd.DataFrame(product_data[0], columns=product_data[1])

retailer_data = db.get_data_with_headers(table_name='FR_HL_RETAIL_V', size=1)
retailer_data = pd.DataFrame(retailer_data[0], columns=retailer_data[1])

retailer1_data = db.get_data_with_headers(table_name='FR_HL_RETAILER1_V', size=1)
retailer1_data = pd.DataFrame(retailer1_data[0], columns=retailer1_data[1])

retailer2_data = db.get_data_with_headers(table_name='FR_HL_RETAILER2_V', size=1)
retailer2_data = pd.DataFrame(retailer2_data[0], columns=retailer2_data[1])

retailer3_data = db.get_data_with_headers(table_name='FR_HL_RETAILER3_V', size=1)
retailer3_data = pd.DataFrame(retailer3_data[0], columns=retailer3_data[1])

In [22]:
#Request data
data = {
    'dist_data': dist_data.values.tolist(),
    'product_data': product_data.values.tolist(),
    'retailer_data': retailer_data.values.tolist(),
    'retailer1_data': retailer1_data.values.tolist(),
    'retailer2_data': retailer2_data.values.tolist(),
    'retailer3_data': retailer3_data.values.tolist()
}
test_data = json.dumps(data)

In [24]:
result = predict(endpoint_url=endpoint_url[0],compute_type='local',data=test_data)

2023-12-15 11:28:29,275: fedml_azure.logger INFO: Using the parameters 'endpoint_url' and 'compute_type' for inferencing.


In [25]:
scores = result['result']
len(scores)

122400

In [26]:
type(scores)

list

In [27]:
result_df = pd.DataFrame(scores, columns=['retailer', 'productsku', 'calendar_year','calendar_month', 'Predictions'])

In [28]:
types = {
    'retailer': 'int',
    'productsku': 'int',
    'calendar_year': 'int',
    'calendar_month': 'int'
}
result_df = result_df.astype(types)

In [29]:
result_df['ID'] = result_df.index

In [30]:
result_df.head(10)

,retailer,productsku,calendar_year,calendar_month,Predictions,ID
0,10001,1061,2019,1,1618.0,0
1,10001,1061,2019,2,1101.0,1
2,10001,1061,2019,3,4229.0,2
3,10001,1061,2019,4,4641.0,3
4,10001,1061,2019,5,3180.0,4
5,10001,1061,2019,6,3485.0,5
6,10001,1061,2019,7,497.0,6
7,10001,1061,2019,8,4085.0,7
8,10001,1061,2019,9,666.0,8
9,10001,1061,2019,10,2815.0,9


In [50]:
db.create_table("CREATE TABLE FR_RL_Retail_Predictions (ID INTEGER PRIMARY KEY, retailer INTEGER, productsku INTEGER, calendar_year INTEGER, calendar_month INTEGER, Predictions FLOAT(2))")

2023-12-15 10:05:24,916: fedml_azure.logger INFO: creating table...
2023-12-15 10:05:24,917: fedml_azure.logger INFO: CREATE TABLE FR_RL_Retail_Predictions (ID INTEGER PRIMARY KEY, retailer INTEGER, productsku INTEGER, calendar_year INTEGER, calendar_month INTEGER, Predictions FLOAT(2), INSERTED_AT TIMESTAMP NOT NULL)
2023-12-15 10:05:24,926: fedml_azure.logger ERROR: error occured during query execution, doing rollback (288, 'cannot use duplicate table name: FR_RL_RETAIL_PREDICTIONS: line 1 col 14 (at pos 13)')


ProgrammingError: (288, 'cannot use duplicate table name: FR_RL_RETAIL_PREDICTIONS: line 1 col 14 (at pos 13)')

In [55]:
db.insert_into_table('FR_RL_RETAIL_PREDICTIONS',result_df)

2023-12-15 10:08:06,608: fedml_azure.logger INFO: inserting into table...
2023-12-15 10:08:06,609: fedml_azure.logger INFO:  INSERT INTO "FR_RL_RETAIL_PREDICTIONS"  (retailer, productsku, calendar_year, calendar_month, Predictions, ID, INSERTED_AT) VALUES (:retailer, :productsku, :calendar_year, :calendar_month, :Predictions, :ID, :INSERTED_AT)
